In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
# Define constants
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 50

In [3]:
# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [4]:
# Get the class names
class_names = dataset.class_names

In [5]:
# Split the dataset into train, validation, and test sets
train_ds = dataset.take(54)
val_ds = dataset.skip(54).take(6)
test_ds = dataset.skip(60)


In [6]:
# Data augmentation and preprocessing
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1./255),
])

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)

In [7]:
# Define the CNN model
cnn_model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

In [8]:
# Define the ANN model
ann_model = models.Sequential([
    resize_and_rescale,
    layers.Flatten(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

In [9]:
# Compile the CNN and ANN models
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
# Train the CNN and ANN models
cnn_history = cnn_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=1)
ann_history = ann_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=1)


Epoch 1/50
54/54 [==============================] - 84s 1s/step - loss: 9.3981 - accuracy: 0.6256 - val_loss: 0.6356 - val_accuracy: 0.7917
Epoch 2/50
54/54 [==============================] - 79s 1s/step - loss: 0.3241 - accuracy: 0.8767 - val_loss: 0.3138 - val_accuracy: 0.8438
Epoch 3/50
54/54 [==============================] - 78s 1s/step - loss: 0.2505 - accuracy: 0.8964 - val_loss: 0.2613 - val_accuracy: 0.9115
Epoch 4/50
54/54 [==============================] - 76s 1s/step - loss: 0.2710 - accuracy: 0.8981 - val_loss: 0.3267 - val_accuracy: 0.8958
Epoch 5/50
54/54 [==============================] - 76s 1s/step - loss: 0.2130 - accuracy: 0.9167 - val_loss: 0.4273 - val_accuracy: 0.8854
Epoch 6/50
54/54 [==============================] - 76s 1s/step - loss: 0.2069 - accuracy: 0.9207 - val_loss: 0.3447 - val_accuracy: 0.8646
Epoch 7/50
54/54 [==============================] - 75s 1s/step - loss: 0.2012 - accuracy: 0.9207 - val_loss: 0.8027 - val_accuracy: 0.7083
Epoch 8/50
54/54 [==

In [11]:
# Create a list to store the trained models
models = [cnn_model, ann_model]


In [12]:
# Define a function to combine the predictions of the individual models
def ensemble_predict(models, image):
  # Get the predictions of each model.
  predictions = []
  for model in models:
    prediction = model.predict(image)
    predictions.append(prediction)

  # Combine the predictions using a weighted averaging method.
  combined_prediction = tf.reduce_mean(predictions, axis=0)

  # Return the combined prediction.
  return combined_prediction

In [13]:
# Make predictions on the test set using the ensemble model
test_ds = test_ds.map(resize_and_rescale)
test_predictions = ensemble_predict(models, test_ds)

8/8 [==============================] - 1s 22ms/step


In [14]:
# Get the predicted classes
test_predicted_classes = tf.argmax(test_predictions, axis=1)